In [1]:
## Setting up the example:
import os
os.chdir("/Users/vtshitoyan/dev/TRI/propnet")

In [2]:
from propnet import Propnet
from propnet import Material
from propnet.symbols import SymbolType
from propnet import Symbol
p = Propnet()

salt = Material()
# refractive_index = Symbol(SymbolType['refractive_index'], 1.49, [])
# extinction_coefficient = Symbol(SymbolType['extinction_coefficient'], 0.2, [])
relative_permittivity = Symbol(SymbolType['relative_permittivity'], 3, [])
refractive_index = Symbol(SymbolType['refractive_index'], 1.4, [])
relative_permittivity_im = Symbol(SymbolType['relative_permittivity_im'], 0.2, [])
absorption_coefficient = Symbol(SymbolType['absorption_coefficient'], 7*10**(-4), []) # in m^-1
wavelength = Symbol(SymbolType['wavelength'], 1060, []) # wavelength in nm
thickness = Symbol(SymbolType['thickness'], 10, []) # thickness in nm

# salt.add_property(refractive_index)
salt.add_property(relative_permittivity)
salt.add_property(relative_permittivity_im)
salt.add_property(thickness)
# salt.add_property(absorption_coefficient)
salt.add_property(wavelength)

p.add_material(salt)
p.evaluate(material=salt)
print(salt)

Failed to parse lame_first_parameter.yaml, 'ascii' codec can't decode byte 0xc3 in position 81: ordinal not in range(128).
'ascii' codec can't decode byte 0xc3 in position 626: ordinal not in range(128)
'ascii' codec can't decode byte 0xcf in position 176: ordinal not in range(128)
'ascii' codec can't decode byte 0xce in position 258: ordinal not in range(128)
'ascii' codec can't decode byte 0xc2 in position 529: ordinal not in range(128)
Material: eef1c650-13d3-46a4-bc82-629037e274c4
	relative_permittivity:	3
	relative_permittivity_im:	0.2
	thickness:	10
	wavelength:	1060
	extinction_coefficient:	0.0577030141092359
	refractive_index:	1.73301172466815



In [3]:
p.evaluate(material=salt)
print(salt)

Material: eef1c650-13d3-46a4-bc82-629037e274c4
	relative_permittivity:	3
	relative_permittivity_im:	0.2
	thickness:	10
	wavelength:	1060
	extinction_coefficient:	0.0577030141092359
	refractive_index:	1.73301172466815
	relative_permeability:	1.00110987927910
	reflectance:	0.0723482912674424
	absorption_coefficient:	0.000684073076285146



In [3]:
SymbolType['thickness'].value.as_dict()

{'@class': 'SymbolMetadata',
 '@module': 'propnet.core.symbols',
 'comment': 'thickness of the material',
 'dimension': 1,
 'display_names': ['Thickness'],
 'display_symbols': ['t'],
 'name': 'thickness',
 'test_value': 100,
 'type': 'condition',
 'units': <Quantity(1.0, 'nanometer')>}

In [8]:
from propnet.models import *
model = OpticalAbsorbance()
print(model.title)
print(model.description)
print(model.name)
print(model.equations)
print(model.symbol_mapping)
print(model.connections)

Optical Absorbance from the absorption coefficient, thickness and reflectance

Absorbance A quantifies the reduction of light intensity as it passes through the material,
i.e. I = I_0(1 - R - A), where I_0 is the intensity at the surface and I is the intensity of
the light that comes out. R is the reflectance, which is the proportion of the reflected light
at the interface.

Frequency-dependent absorption coefficient alpha models the rate of the exponential decay of
intensity as the light passes through medium, and is derived from the extinction coefficient k.
OpticalAbsorbance
['A - (1 - R) * (1 - exp(-alpha * t))']
{'alpha': 'absorption_coefficient', 't': 'thickness', 'R': 'reflectance', 'A': 'absorbance', 'T': 'transmittance'}
[{'inputs': ['alpha', 't', 'R'], 'outputs': 'A'}, {'inputs': ['A', 't', 'R'], 'outputs': 'alpha'}]


In [11]:
model.evaluate({'t': 100, 'alpha': 0.007, 'R': 0.2})

{'A': 0.402731756966872, 'successful': True}

In [7]:
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr

symbol_values = {'t': 100, 'alpha': 0.007, 'R': 0.2}
expr = ["A - (1 - R) * (1 - exp(-alpha * t))"]
eqn = [parse_expr(exp) for exp in expr]
eqn = [eq.subs(symbol_values).evalf() for eq in eqn]

possible_outputs = set()
for eq in eqn:
    possible_outputs = possible_outputs.union(eq.free_symbols)
outputs = {} 


for i, possible_output in enumerate(possible_outputs):
    solutions = sp.nonlinsolve(eqn, possible_output)
    solution = list(solutions)[0][0]
    if not isinstance(solution, sp.EmptySet):
        outputs[str(possible_output)] = solution
outputs

{'A': 0.402731756966872}

In [4]:
from propnet.models import *
model = ComplexRefrFromComplexPerm()
print(model.title)
print(model.description)
print(model.name)
print(model.equations)
print(model.symbol_mapping)
print(model.connections)

Refraction Index and Extinction Coefficient from Comlex Permittivity

Assumes that the magnetic permeability \mu is 1
ComplexRefrFromComplexPerm
['n - sqrt((sqrt(e1**2+e2**2)+e1)/2)', 'k - sqrt((sqrt(e1**2+e2**2)-e1)/2)']
{'e1': 'relative_permittivity', 'e2': 'relative_permittivity_im', 'n': 'refractive_index', 'k': 'extinction_coefficient'}
[{'inputs': ['e1', 'e2'], 'outputs': ['n', 'k']}]


In [8]:
model.evaluate({'e1': 1.5, 'e2': 0.2})

{'k': 0.0814696110281483, 'n': 1.22745154589543, 'successful': True}

In [12]:
from propnet.models import *
model = OpticalAbsorptionCoefficient()
print(model.title)
print(model.description)
print(model.name)
print(model.equations)
print(model.symbol_mapping)
print(model.connections)

Optical Absorption coefficient alpha from extinction coeff and wavelength

Absorption coefficient characterizes the decay of intensity as light goes through the material.
It scales linearly with frequency and extinction coefficient (imaginary part of relative permittivity).

OpticalAbsorptionCoefficient
['alpha - (4*pi*k*l**-1)']
{'alpha': 'absorption_coefficient', 'k': 'extinction_coefficient', 'l': 'wavelength'}
[{'inputs': ['k', 'l'], 'outputs': 'alpha'}, {'inputs': ['alpha', 'l'], 'outputs': 'k'}, {'inputs': ['k', 'alpha'], 'outputs': 'l'}]


In [16]:
model.evaluate({'k': 0.1, 'l': 1060})

{'alpha': 0.00118550666173200, 'successful': True}

In [2]:
from propnet.models import *
model = RefractiveIndexfromRelPerm()
print(model.title)
print(model.description)
print(model.name)
print(model.equations)
print(model.symbol_mapping)
print(model.connections)

Failed to parse lame_first_parameter.yaml, 'ascii' codec can't decode byte 0xc3 in position 81: ordinal not in range(128).
Refractive index, relative permeability and permittivity

The refractive index gives the factor by which the speed of light is reduced in a medium.

Likewise, modeling the induced magnetic and electric dipoles as linear within a material,
a relative spatial electrical permittivity and relative spatial magnetic permeability
arise from consideration of the total electrical and magnetic fields.

From the Maxwell Relations, the index of refraction is equal to the geometric mean  of the
relative permittivity and the relative permeability.

RefractiveIndexfromRelPerm
['n - sqrt(Ur*Er)']
{'Er': 'relative_permeability', 'Ur': 'relative_permittivity', 'n': 'refractive_index'}
[{'inputs': ['Ur', 'Er'], 'outputs': 'n'}, {'inputs': ['Er', 'n'], 'outputs': 'Ur'}, {'inputs': ['Ur', 'n'], 'outputs': 'Er'}]


In [3]:
model.evaluate({'n': 1, 'Ur': 1})

{'Er': ConditionSet(Er, Eq(Er**0.5 - 1.0, 0), Complexes(S.Reals x S.Reals, False)),
 'successful': True}